*   Network     : Inception v2
*   Optimizer   : ADAM
*   Regularizer : Dropout

Ref for code: https://arxiv.org/pdf/1512.00567v3.pdf

In [ ]:
from __future__ import print_function

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras import callbacks

from keras.applications.imagenet_utils import preprocess_input
from keras.datasets import cifar100
from keras.initializers import glorot_uniform
from keras.layers.core import Lambda
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, concatenate, Dropout
from keras.models import Model, load_model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import layer_utils, data_utils, vis_utils, plot_model #data_utils.get_file #vis_utils.model_to_dot

import numpy as np
import pydot
import scipy.misc
import matplotlib.pyplot as plt
import tensorflow as tf

from IPython.display import SVG
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [ ]:
#Loading cifar100 dataset and dividing into Train, Validation and Test sets

(X_train, Y_train), (x_test, y_test_o) = cifar100.load_data()

X_train = X_train.astype('float32')
x_test = x_test.astype('float32')

x_train, x_validation, y_train, y_validation = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

y_train = keras.utils.to_categorical(y_train, 100)
y_validation = keras.utils.to_categorical(y_validation, 100)
y_test = keras.utils.to_categorical(y_test_o, 100)

In [ ]:
weight_decay = 0.0005
num_of_classes = 100
conv_activation_func = 'elu'

l_rate = 0.0001
batch_size = 128
maxepoches = 40
lr_decay = 1e-6

In [ ]:
def inceptionv2(input):
    model = Sequential()

    ##### MAIN LAYER #####
    main = Conv2D(64, (3, 3), strides=(2,2), padding='same',activation=conv_activation_func)(input)
    main = Conv2D(64, (3, 3), strides = (1,1), padding='same', activation=conv_activation_func)(main)
    main = Conv2D(64, (3, 3), strides = (1,1), padding='same', activation=conv_activation_func)(main)

    main = MaxPooling2D(pool_size=(2, 2))(main)

    main = Conv2D(64, (3, 3), strides = (1,1), padding='same', activation=conv_activation_func)(main)
    main = Conv2D(64, (3, 3), strides = (2,2), padding='same', activation=conv_activation_func)(main)
    main = Conv2D(64, (3, 3), strides = (1,1), padding='same', activation=conv_activation_func)(main)

    ######Inception module A#####
    main = inception_A(main)
    main = inception_A(main)
    main = inception_A(main)

    ######Inception module B#####
    main = inception_B(main)
    main = inception_B(main)
    main = inception_B(main)
    main = inception_B(main)
    main = inception_B(main)

    ######Inception module C#####
    main = inception_C(main)
    main = inception_C(main)

    main = AveragePooling2D(pool_size=(3, 3))(main)

    main = Flatten()(main)
    dense = Dense(10, activation='linear')(main)
    dense = Dropout(0.2)(dense)
    output = Dense(num_of_classes, activation='softmax')(dense)
    return output

In [ ]:
def inception_A(input):
  pll1 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(input)
  pll1 = Conv2D(64, (3,3), padding='same', activation=conv_activation_func)(pll1)
  pll1 = Conv2D(64, (3,3), padding='same', activation=conv_activation_func)(pll1)

  pll2 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(input)
  pll2 = Conv2D(64, (3,3), padding='same', activation=conv_activation_func)(pll2)
  
  pll3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input)

  pll3 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(pll3)
  
  pll4 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(input)

  output = keras.layers.concatenate([pll1, pll2, pll3, pll4], axis=3)

  return output

In [ ]:
def inception_B(input):
  pll1 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(input)
  pll1 = Conv2D(64, (1,3), padding='same', activation=conv_activation_func)(pll1)
  pll1 = Conv2D(64, (3,1), padding='same', activation=conv_activation_func)(pll1)
  pll1 = Conv2D(64, (1,3), padding='same', activation=conv_activation_func)(pll1)
  pll1 = Conv2D(64, (3,1), padding='same', activation=conv_activation_func)(pll1)

  pll2 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(input) 
  pll2 = Conv2D(64, (1,3), padding='same', activation=conv_activation_func)(pll2)
  pll2 = Conv2D(64, (3,1), padding='same', activation=conv_activation_func)(pll2)

  pll3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input)
  
  pll3 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(pll3)

  pll4 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(input)

  output = keras.layers.concatenate([pll1, pll2, pll3, pll4], axis = 3)

  return output

In [ ]:
def inception_C(input):
  pll1 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(input)
  pll1 = Conv2D(64, (3,3), padding='same', activation=conv_activation_func)(pll1)

  pll1_a = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(pll1)
  pll1_b = Conv2D(64, (3,3), padding='same', activation=conv_activation_func)(pll1)

  pll2 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(input)
  
  pll2_a = Conv2D(64, (1,3), padding='same', activation=conv_activation_func)(pll2)
  pll2_b = Conv2D(64, (3,1), padding='same', activation=conv_activation_func)(pll2)

  pll3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input)
  pll3 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(pll3)

  pll4 = Conv2D(64, (1,1), padding='same', activation=conv_activation_func)(input)

  output = concatenate([pll1_a, pll1_b, pll2_a, pll2_b, pll3, pll4], axis=3)
  return output

In [ ]:
#Definition of SGD optimizer
adam = optimizers.Adam(learning_rate=l_rate)#, decay=lr_decay,momentum=0.9, nesterov=True)

input = Input(shape=(32, 32, 3))
output = inceptionv2(input)

model = Model([input], output)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [ ]:
#Defintion of callbacks with EarlyStopping 
callback = callbacks.EarlyStopping(monitor='val_loss', patience=10)

#Definition of saving the best weights of the model with ModelCheckpoint
checkpoint_path = "checkpoints"
checkpoint = callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1, mode='min')

#Training the model in the training dataset
history = model.fit(x_train, y_train, epochs = maxepoches, batch_size = batch_size, validation_data=(x_validation, y_validation), callbacks=[callback, checkpoint])
model.save_weights('InceptionV2_ADAM_DropOut.h5')

In [ ]:
#Plot the Training and Validation accuracy v/s Epochs
#history.keys() = ['accuracy', 'loss', 'val_accuracy', 'val_loss']

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#Plot the Training and Validation loss v/s Epochs

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#Testing the model using the best weights of the model

model.load_weights('./InceptionV2_ADAM_DropOut.h5')
loss, acc = model.evaluate(x_test,  y_test, verbose=2)

predict_y_test_o = model.predict(x_test, verbose=0)

In [ ]:
# #Precision, recall and accuracy of the model on test data
predict_y_test = np.argmax(predict_y_test_o, axis=1)

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test_o, predict_y_test)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(y_test_o, predict_y_test, average='macro')
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(y_test_o, predict_y_test, average='macro')
print('Recall: %f' % recall)

# Confusion matrix
confusion_matrix_ = confusion_matrix(y_test_o, predict_y_test)
print(confusion_matrix_)

In [ ]:
import seaborn as sb
heat_map = sb.heatmap(confusion_matrix_)
plt.show()